In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")


@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [3]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [4]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role
    
    if user_role == "internal":
        pass # internal users get access to all tools
    else:
        tools = [web_search] # external users only get access to web search
        request = request.override(tools=tools) 

    return handler(request)

In [5]:
from langchain.agents import create_agent
from langchain_aws import ChatBedrock

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
# Ideal para agentes que necesitan planificar pasos lógicos.
# llm = ChatBedrock(
#     model_id="us.deepseek.r1-v1:0",  # ID oficial validado
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.6, # DeepSeek recomienda 0.6 para razonamiento
#         "max_tokens": 8192,  # Recomendado para no degradar calidad del CoT
#         "top_p": 0.95,
#     }
# )


# llm = ChatBedrock(
#     model_id="us.deepseek.v3-v1:0", # Prueba este primero
#     region_name="us-east-1",        # O us-west-2
#     model_kwargs={
#         "temperature": 0.7,
#         "max_tokens": 4096
#     }
# )
# from langchain_aws import ChatBedrock
# llm = ChatBedrock(
# model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
# # model_id="cohere.command-r-plus-v1:0",
# region_name="us-east-1",
# model_kwargs={
# "temperature": 0.5,
# "max_tokens": 2048,
# "top_p": 0.9,
# }
# )


# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

llm = ChatBedrock(
    model_id="amazon.nova-lite-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [7]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "external"}
)

print(response["messages"][-1].content)

<thinking>I have found several sources that mention the number of artists in a database, but they are all different. The first source mentions a database of 16,000 artists used to train Midjourney AI, but it doesn't specify if this is the total number of artists in the database. The second source mentions a database of 370,062 artists, but it's unclear if this is the total number of artists in the database or just the number of artists in a specific category. The third source mentions a database of over 1.4 million musical artists, but it's unclear if this is the total number of artists in the database or just the number of artists in a specific category. The fourth source mentions a music library with 1,312 artists, but it's unclear if this is the total number of artists in the database or just the number of artists in a specific category. The fifth source mentions a database with 2.7 million artists, but it's unclear if this is the total number of artists in the database or just the 

In [8]:
from IPython.display import display, HTML, Markdown

display(Markdown(response["messages"][-1].content))

<thinking>I have found several sources that mention the number of artists in a database, but they are all different. The first source mentions a database of 16,000 artists used to train Midjourney AI, but it doesn't specify if this is the total number of artists in the database. The second source mentions a database of 370,062 artists, but it's unclear if this is the total number of artists in the database or just the number of artists in a specific category. The third source mentions a database of over 1.4 million musical artists, but it's unclear if this is the total number of artists in the database or just the number of artists in a specific category. The fourth source mentions a music library with 1,312 artists, but it's unclear if this is the total number of artists in the database or just the number of artists in a specific category. The fifth source mentions a database with 2.7 million artists, but it's unclear if this is the total number of artists in the database or just the number of artists in a specific category. I need to find a source that clearly states the total number of artists in the database.</thinking>